In [2]:
with open("data.txt", "r") as f:
    lines = f.readlines()
lines[-1] = lines[-1]+"\n"
f.close()

valveDict = {}

for line in lines:
    valveDict[line[6:8]] = (int(line[line.find("=")+1:line.find(";")]), [x.strip() for x in line[line.find("valve")+6:-1].split(",")])

In [6]:
import functools
import re

In [9]:
@functools.lru_cache(maxsize=None)
def maxflow(current, visited, timeLeft):
    if timeLeft <= 0:
        return 0
    best = 0
    if current not in visited:
        val = (timeLeft - 1) * valveDict[current][0]
        current_opened = tuple(sorted(visited + (current,)))

        for destination in valveDict[current][1]:
            if val != 0:
                best = max(best,
                    val + maxflow(destination, current_opened, timeLeft - 2))
            best = max(best,
                maxflow(destination, visited, timeLeft - 1))
    else:
        for destination in valveDict[current][1]:
            best = max(best,
                maxflow(destination, visited, timeLeft - 1))
    return best

print(f'Max pressure released after 30 mins: {maxflow("AA", (), 30)}')

Max pressure released after 30 mins: 1580


In [28]:
with open("data.txt", "r") as f:
    lines = [re.split('[\\s=;,]+', x) for x in f.read().splitlines()]
f.close()

nodeMap = {x[1]: set(x[10:]) for x in lines}
flows = {x[1]: int(x[5]) for x in lines if int(x[5]) != 0}
I = {x: 1<<i for i, x in enumerate(flows)}
timeTaken = {x: {y: 1 if y in nodeMap[x] else float('+inf') for y in nodeMap} for x in nodeMap}

for k in timeTaken:
    for i in timeTaken:
        for j in timeTaken:
            timeTaken[i][j] = min(timeTaken[i][j], timeTaken[i][k]+timeTaken[k][j])

def visit(v, budget, state, flow, answer):
    answer[state] = max(answer.get(state, 0), flow)
    for u in flows:
        newbudget = budget - timeTaken[v][u] - 1
        if I[u] & state or newbudget <= 0: continue
        visit(u, newbudget, state | I[u], flow + newbudget * flows[u], answer)
    return answer

total1 = max(visit('AA', 30, 0, 0, {}).values())
visited2 = visit('AA', 26, 0, 0, {})

total2 = max(v1+v2 for k1, v1 in visited2.items() 
                   for k2, v2 in visited2.items() if not k1 & k2)
print(total1, total2)

dict_items([(0, 0), (1, 132), (3, 355), (7, 451), (519, 617), (1031, 541), (4103, 645), (8199, 604), (11, 515), (15, 571), (27, 683), (91, 815), (283, 960), (43, 795), (2091, 936), (75, 689), (1099, 731), (8267, 763), (16459, 712), (139, 781), (2187, 995), (267, 779), (331, 901), (2315, 914), (16651, 779), (523, 627), (1035, 578), (2059, 710), (2123, 836), (18443, 728), (8203, 619), (16395, 542), (16411, 636), (16427, 807), (16523, 787), (17419, 584), (18571, 998), (24587, 622), (19, 542), (83, 749), (275, 813), (35, 685), (39, 741), (171, 633), (299, 959), (51, 787), (99, 837), (163, 916), (2211, 1101), (291, 845), (547, 797), (1059, 748), (2083, 856), (2147, 960), (2339, 667), (18467, 865), (8227, 789), (16419, 703), (16483, 857), (16547, 913), (67, 587), (71, 651), (339, 955), (323, 753), (579, 713), (1091, 657), (1095, 713), (1603, 755), (9283, 751), (2115, 764), (4163, 741), (8259, 713), (8263, 765), (8771, 832), (16451, 622), (18499, 790), (131, 691), (395, 609), (195, 811), (387